<a href="https://colab.research.google.com/github/MunYeong0723/NLP_mini-project/blob/master/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_cosin-similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
movie_list = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/list_movie.csv")
movie_list

,code,title,year,genres,ratings_avg
0,md6B3Ad,대부 일대기,1992,"['드라마', '범죄']",4.4
1,mNdKBkO,쇼생크 탈출,1994,['드라마'],4.4
2,mV539wO,인생은 아름다워,1997,"['드라마', '코미디']",4.4
3,mW4LYBo,하이큐!! 콘셉트의 싸움,2017,"['애니메이션', '스포츠']",4.4
4,m5r0pVO,12인의 노한 사람들,1997,"['드라마', '범죄']",4.2
...,...,...,...,...,...
292,mWvDGZd,써니,2011,"['드라마', '코미디']",4.0
293,m5338w5,미녀와 야수,1991,"['애니메이션', '가족', '판타지', '로맨스', '키즈']",4.0
294,m5XMJ1W,노인을 위한 나라는 없다,2007,"['범죄', '스릴러', '드라마']",4.0
295,mVWzLNd,내가 널 사랑할 수 없는 10가지 이유,1999,"['로맨틱코미디', '틴에이저']",4.0


In [ ]:
movie_list['genres'] = movie_list['genres'].apply(literal_eval)
movie_list['genres'] = movie_list['genres'].apply(lambda x : ' '.join(x))
movie_list

,code,title,year,genres,ratings_avg
0,md6B3Ad,대부 일대기,1992,드라마 범죄,4.4
1,mNdKBkO,쇼생크 탈출,1994,드라마,4.4
2,mV539wO,인생은 아름다워,1997,드라마 코미디,4.4
3,mW4LYBo,하이큐!! 콘셉트의 싸움,2017,애니메이션 스포츠,4.4
4,m5r0pVO,12인의 노한 사람들,1997,드라마 범죄,4.2
...,...,...,...,...,...
292,mWvDGZd,써니,2011,드라마 코미디,4.0
293,m5338w5,미녀와 야수,1991,애니메이션 가족 판타지 로맨스 키즈,4.0
294,m5XMJ1W,노인을 위한 나라는 없다,2007,범죄 스릴러 드라마,4.0
295,mVWzLNd,내가 널 사랑할 수 없는 10가지 이유,1999,로맨틱코미디 틴에이저,4.0


### user가 좋게 평가한 영화들의 장르를 찾아서 이와 유사한 영화를 찾음

In [ ]:
rating_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/rating_data_pre.csv")
rating_data

,제목,장르,총 평점,유저 평점
0,남매의 여름밤,드라마,3.9,3.5
1,69세,드라마,3.6,3.5
2,오케이 마담,코미디,2.7,2.5
3,반교: 디텐션,공포,3.4,3.0
4,다만 악에서 구하소서,범죄,3.0,3.0
...,...,...,...,...
5089,다이하드: 굿 데이 투 다이,액션,3.3,3.0
5090,본 얼티메이텀,액션,4.1,4.0
5091,가까이서 본 기차,전쟁,3.6,3.5
5092,디스트릭트 9,SF,3.8,3.5


In [ ]:
like_movie = rating_data[rating_data['유저 평점'] > 3.0]
like_movie = like_movie.sort_values(by=['유저 평점', '총 평점'], axis=0, ascending=False).reset_index()
like_movie

,index,제목,장르,총 평점,유저 평점
0,1758,철서구 2부 - 폐허,다큐멘터리,4.4,4.0
1,3245,인생은 아름다워,드라마,4.4,4.0
2,3259,쇼생크 탈출,드라마,4.4,4.0
3,4110,인셉션,액션,4.4,4.0
4,173,기생충,드라마,4.3,4.0
...,...,...,...,...,...
1663,4963,슈렉 2,판타지,3.5,3.5
1664,4974,소셜 네트워크,드라마,3.5,3.5
1665,5043,래빗 홀,드라마,3.5,3.5
1666,5068,내일의 기억,드라마,3.5,3.5


In [ ]:
user_like = ' '.join(like_movie['장르'].unique())
tmp = movie_list.append({'genres':user_like}, ignore_index=True)

tfidf_vec = TfidfVectorizer(ngram_range=(1,1))
new_matrix = tfidf_vec.fit_transform(tmp['genres'])

new_similarity = cosine_similarity(new_matrix, new_matrix)
new_similarity_index = np.argsort(-new_similarity)
new_similarity_index.shape

(298, 298)

In [ ]:
user_index = [len(movie_list)]
new_similar_movies = new_similarity_index[user_index, 1:]
new_similar_movies_index = new_similar_movies.reshape(-1)

similar_movie = movie_list.iloc[new_similar_movies_index].reset_index()
similar_movie

,index,code,title,year,genres,ratings_avg
0,124,mWvkXvZ,코코,2017,애니메이션 모험 가족 판타지 음악 미스터리 코미디 스릴러,4.0
1,162,mXOgMN5,바람과 함께 사라지다,1939,전쟁 로맨스 시대극 드라마 코미디 역사,4.0
2,238,mW93a1d,라푼젤,2010,로맨스 가족 판타지 코미디 애니메이션 모험 키즈,4.0
3,128,m45nnY5,월-E,2008,가족 모험 코미디 SF 애니메이션 키즈,4.1
4,282,mW4LAwW,라따뚜이,2007,판타지 가족 드라마 코미디 애니메이션 모험 키즈,4.0
...,...,...,...,...,...,...
292,240,mOo0AGa,자니 익스프레스,2014,애니메이션 단편,4.0
293,158,mdjanY9,위플래쉬 (단편),2013,드라마 단편,4.0
294,181,m533xZ3,몸 값,2015,단편 드라마,3.9
295,18,m5r0koO,타이타닉,1996,,4.2


### 유사한 영화 중 보고 싶은 영화 리스트에 있는 영화 5-10개 추천
(리스트가 없거나 부족하면 나머지 list 가져와서 갯수 맞추기)

In [ ]:
wish_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/wish_data_pre.csv")
wish_data

,제목,장르,총 평점
0,SPF-18,로맨스,1.5
1,걸즈 트립,코미디,3.2
2,무민 더 무비,가족,3.4
3,사샤의 북극 대모험,애니메이션,3.8
4,키싱 부스 2,로맨스,2.8
...,...,...,...
5308,"헤일, 시저!",코미디,3.4
5309,컨버세이션,범죄,3.8
5310,사울의 아들,드라마,3.8
5311,잭 리처: 네버 고 백,범죄,2.9


In [ ]:
wish_data_sorted = wish_data.sort_values(by='총 평점', ascending=False)
wish_data_sorted = wish_data_sorted.drop_duplicates(['제목']).reset_index()
wish_data_sorted.columns = ['index', 'title', 'genres', 'rating_avg']
wish_data_sorted

,index,title,genres,rating_avg
0,4710,컷 포이슨 번,다큐멘터리,4.5
1,2058,인생은 아름다워,드라마,4.4
2,409,한나 개즈비: 나의 이야기,코미디,4.4
3,4378,쇼생크 탈출,드라마,4.4
4,2519,인셉션,액션,4.4
...,...,...,...,...
4060,5193,핫 봇,코미디,1.3
4061,3646,드래곤볼 에볼루션,액션,1.2
4062,1295,키스 미 어게인,미국,1.1
4063,5122,리얼,액션,1.1


In [ ]:
recommend = pd.merge(similar_movie.loc[:, ['title','year', 'genres', 'ratings_avg']], wish_data_sorted.loc[:, ['title']], on='title')
recommend.sort_values(by='ratings_avg', ascending=False).reset_index()[:10]

,index,title,year,genres,ratings_avg
0,217,쇼생크 탈출,1994,드라마,4.4
1,195,인생은 아름다워,1997,드라마 코미디,4.4
2,51,인셉션,2010,액션 모험 SF 스릴러,4.4
3,210,레미제라블 25주년 런던 라이브 공연,2010,드라마,4.3
4,166,타이타닉,1997,로맨스 드라마,4.3
5,126,포레스트 검프,1994,드라마 로맨스 코미디,4.3
6,19,어바웃 타임,2013,판타지 드라마 로맨틱코미디 코미디 로맨스,4.3
7,230,시네마 천국,1988,드라마,4.3
8,127,대부,1972,범죄 드라마 스릴러,4.3
9,212,기생충,2019,드라마,4.3
